<center>
<h1 style="color: pink;">Welcome to Publish Mate 😊</h1>
</center>

## `00` Download Dependencies

In [20]:
# !pip3 install -U "crewai[tools,agentops]"

In [21]:
# !pip3 install python-dotenv
# !pip3 install gcloud
# !pip3 install google-genai

## `01` Import Libraries

In [22]:
from dotenv import load_dotenv
import os
import google.generativeai as genai

In [ ]:
from crewai import Agent, Task, Crew, Process, LLM
from crewai.tools import tool
from crewai.knowledge.source.string_knowledge_source import StringKnowledgeSource
from crewai.llms.base_llm import BaseLLM

from pydantic import BaseModel, Field, HttpUrl
from typing import List, Dict

import agentops
import json
import gcloud
import google.generativeai as genai
from dotenv import load_dotenv
from vertexai.preview.generative_models import Content, Part

from tavily import TavilyClient
from crewai.tools import tool


## `02` load api key

In [24]:
load_dotenv()  # Load from .env

True

In [25]:
AGENTOPS_API_KEY = os.getenv("AGENTOPS_API_KEY") # replace by yours

In [92]:
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")
PROJECT_ID = os.getenv("PROJECT_ID")
PROJECT_NAME = os.getenv("PROJECT_NAME")

TAVILY_API_KEY = os.getenv("TAVILY_API_KEY")

# SERPER_API_KEY = os.getenv("SERPERDEV_API_KEY")
# os.environ["SERPER_API_KEY"] = SERPER_API_KEY
# os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

genai.configure(api_key=OPENAI_API_KEY)

## `03` Start AgentOps session

In [96]:
agentops.init(api_key=AGENTOPS_API_KEY,
               skip_auto_end_session=True, # Set to True to skip auto ending the session
               default_tags=['crewai']
               ) 

The link will help us to monitor our agents

### Make sure it works

In [28]:
# print("AgentOps session initialized.")
# print(agentops.session)  # optional, shows session info if available
# print(agentops.__dict__)

## `04` Intro of the Crew

In [29]:
intro_prompt = (
    "Welcome to PublishMate! I am your research assistant mate here to help you with your academic paper journey.\n"
    "I will guide you step-by-step to find trending topics, recent papers, summaries, "
    "research gaps, and help with paper writing. \nLet's get started!\n"
)

def welcome_message():
    print(intro_prompt)

# Run this at the very beginning
welcome_message()

Welcome to PublishMate! I am your research assistant mate here to help you with your academic paper journey.
I will guide you step-by-step to find trending topics, recent papers, summaries, research gaps, and help with paper writing. 
Let's get started!



## `05` Set Output dir

In [30]:
output_dir = './PublishMate_agent_ouput'
os.makedirs(output_dir, exist_ok=True)

## `06` LLM will be used

In [31]:
basic_llm = LLM(
    model="gemini/gemini-1.5-flash",
    temperature=0.2,
    provider="google_ai_studio",
    api_key=os.environ["GEMINI_API_KEY"]
)

## `07` START AGENTS

### `7.1` Agent 1: Trending Topics Agent 

In [32]:
# !gcloud init

In [33]:
user_input = input("Enter your research field or keyword: ")

In [34]:
class TrendingTopicsOutput(BaseModel):
    topics: List[Dict[str, str]] = Field(..., title="Trending topics with description", min_items=1)

trending_topics_agent = Agent(
    role="Trending Topics Identification Agent",

    goal="\n".join([
        f"You are an expert research assistant that identifies the latest trending topics in the field of {user_input} only focus on it .",
        "Generate a detailed list of the top 3-5 trending topics or recent articles reflecting advances and high interest in this field.",
        "Base your answer on recent publication trends, conferences, or journal articles.",
        "Do not include unrelated or general topics.",
        "Output only a JSON object with a 'topics' list containing objects with 'name' and 'description'."
    ]),
    backstory="Designed to guide users by providing the most relevant and current trending research topics in their specified field.",
    llm=basic_llm,
    verbose=True,
)

trending_topics_task = Task(
    description="\n".join([
        f"you are an expert in a {user_input} field to help beginner researchers in their writings .",
        "Provide a list of 3 to 5 trending topics or articals with a brief description for each.",
        "Focus on recent research interests supported by publication trends.",
        "Output in JSON format with 'topics' as list of objects {name, description}."
    ]),
    expected_output="JSON object with list of trending topics and descriptions.",
    output_json=TrendingTopicsOutput,
    output_file=os.path.join(output_dir, "step_1_trending_topics.json"),
    agent=trending_topics_agent,
)

/home/israa/Desktop/PublishMate_CrewAgents/myenv/lib/python3.10/site-packages/pydantic/fields.py:1058: PydanticDeprecatedSince20: `min_items` is deprecated and will be removed, use `min_length` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  warn('`min_items` is deprecated and will be removed, use `min_length` instead', DeprecationWarning)


### `7.2` Agent 2: Recent Papers Retrieval Agent

In [110]:

search_client = TavilyClient(api_key=TAVILY_API_KEY)
 

# tavily_client = TavilyClient(api_key=TAVILY_API_KEY)

@tool
def search_engine_tool(query: str):
    """Useful for search-based queries. Use this to find current information about any query related pages using a search engine"""
    return search_client.search(query)


In [111]:
class PaperInfo(BaseModel):
    title: str 
    authors: List[str]  # List of authors as strings
    abstract: str                                   
    year: int # = Field(..., ge=2023)               # Year must be 2023 or later
    url: str 

class RecentPapersOutput(BaseModel):
    topic_papers: Dict[str, List[PaperInfo]] = Field(..., title="Recent papers grouped by topic")

recent_papers_agent = Agent(
    role="Recent Papers Retrieval Agent",

    goal = "\n".join([
        "You are a research paper search assistant.",
        "Given a list of trending topics, retrieve 3 recent, relevant publications per topic.",
        "Select papers from reputable sources published within the last 2 years.(2023 or 2024 or 2025)",
        "Provide title, authors, abstract, year, and valid URL for each paper.",
        "the URL must be valid and accessible.",
        "If no recent paper is available, state 'No recent papers found' for that topic.",
        "Output in JSON format grouped by topic."]),

    backstory="Helps beginner researchers quickly discover and review the latest relevant publications across the trending topics with the URLs that are valid and some info.",

    llm=basic_llm,
    
    verbose=True,
)

recent_papers_task = Task(
    description="\n".join([
        "Input is a list of trending topics.",
        "For each topic, find 3 papers with title, authors, abstract, year, and link which should be valid and accessable.",
        "Select papers from reputable journals or conferences (IEEE, Springer, Elsevier, ICRA, IROS, actual arXiv).",
        "Only include papers published in 2023 or 2024 or 2025.",
        "Focus on papers from last 2 years from reputable conferences or journals.",
        "If no recent paper is available, state 'No recent papers found' for that topic.",
        "Output JSON grouped by topic."
    ]),
    expected_output="JSON with topics as keys and list of paper info objects as values.",
    output_json=RecentPapersOutput,
    output_file=os.path.join(output_dir, "step_2_recent_papers.json"),
    agent=recent_papers_agent,
    tools=[search_engine_tool],
    
)


### `7.3` Agent 3: Paper Summarization Agent

In [ ]:
class PaperSummariesOutput(BaseModel):
    summaries: Dict[str, str] = Field(
        ..., 
        title="Paper title mapped to its summary", 
        description="Each item has 'title' and 'summary'."
    )

paper_summarization_agent = Agent(
    role="Academic Paper Summarization Agent",
    goal="\n".join([
        "Summarize each research paper into a detailed 120-150 word paragraph.",
        "Mention the full paper title before the summary.",
        "Focus on: main research problem, methodology, key findings, unique contributions.",
        "Highlight any datasets, models, or diagrams used (in the paper).",
        "Avoid generic descriptions. Be specific about what the paper achieves."
    ]),
    backstory="Provides clear and informative summaries to help users understand research papers quickly even if they are beginners.",
    llm=basic_llm,
    verbose=True,
)

paper_summarization_task = Task(
    description="\n".join([
        "Input is a list of papers with metadata and abstracts.",
        "Produce a summary for each paper highlighting key points and visuals if any.",
        "Output JSON mapping paper titles to summaries."
    ]),
    expected_output="JSON object mapping paper titles to summaries.",
    output_json=PaperSummariesOutput,
    output_file=os.path.join(output_dir, "step_3_paper_summaries.json"),
    agent=paper_summarization_agent,
)


### `7.4` Agent 4: Research Gap and Suggestion Agent

In [ ]:
class ResearchGapOutput(BaseModel):
    research_gaps: List[str] = Field(..., title="List of research gaps and suggestions")

research_gap_agent = Agent(
    role="Research Gap Identification and Suggestion Agent",
    goal="\n".join([
        "Analyze summaries to identify gaps, limitations, and propose research directions or improvements.",
        "Use a friendly and encouraging tone suitable for beginners."
    ]),
    backstory="Helps users find novel contributions by highlighting unexplored areas and providing ideas.",
    llm=basic_llm,
    verbose=True,
)

research_gap_task = Task(
    description="\n".join([
        "Input is paper summaries.",
        "Output a list of research gaps, limitations, and suggestions for future research.",
        "Encourage beginners by providing feasible ideas."
    ]),
    expected_output="JSON list of research gaps and improvement suggestions.",
    output_json=ResearchGapOutput,
    output_file=os.path.join(output_dir, "step_4_research_gaps.json"),
    agent=research_gap_agent,
)

### `7.5` Agent 5: Paper Structure and Writing Guide Agent

In [ ]:
class PaperStructureSection(BaseModel):
    section: str
    tips: str

class PaperStructureOutput(BaseModel):
    paper_structure: List[PaperStructureSection] = Field(..., title="Paper structure sections and writing tips")

paper_structure_agent = Agent(
    role="Paper Structure and Writing Guide Agent",
    goal="\n".join([
        "Provide a clear outline for structuring an academic paper.",
        "Give detailed tips on what to write in each section to help beginners.",
        "Include motivational and supportive writing advice."
    ]),
    backstory="Guides users through the paper writing process with a beginner-friendly approach.",
    llm=basic_llm,
    verbose=True,
)

paper_structure_task = Task(
    description="\n".join([
        "Input is the chosen research topic.",
        "Output a recommended paper structure with sections and detailed writing tips for each.",
        "Help beginners understand what content belongs in each part of the paper."
    ]),
    expected_output="JSON list of sections with writing tips.",
    output_json=PaperStructureOutput,
    output_file=os.path.join(output_dir, "step_5_paper_structure.json"),
    agent=paper_structure_agent,
)


### `7.6` Agent 6: Related work draft Agent

In [ ]:
class RelatedWorkOutput(BaseModel):
    related_work: str = Field(..., title="Composed related work section")

related_work_agent = Agent(
    role="Related Work Composer Agent",
    goal="\n".join([
        "Compose a comprehensive 'Related Work' section using the paper summaries.",
        "Organize by themes or trends, and mention each paper's key contributions.",
        "Maintain academic tone and proper citation-like references (e.g., 'Smith et al. 2023')."
    ]),
    backstory="Helps users create strong literature review content automatically.",
    llm=basic_llm,
    verbose=True,
)

related_work_task = Task(
    description="\n".join([
        "Input is the list of paper summaries.",
        "Group papers by similarity and write a flowing Related Work section.",
        "Ensure good transitions, academic tone, and clear references.",
        "Output as a single string."
    ]),
    expected_output="Single string of the Related Work section.",
    output_json=RelatedWorkOutput,
    output_file=os.path.join(output_dir, "step_6_related_work.json"),
    agent=related_work_agent,
)


### `7.7` Agent 7: Paper draft Agent

In [ ]:
class DraftOutput(BaseModel):
    draft: str = Field(..., title="Full academic paper draft text")

draft_writer_agent = Agent(
    role="Academic Paper Drafting Agent",
    goal="\n".join([
        "Write a full academic paper draft using the structure, research gap, and related work.",
        "Ensure clarity, academic tone, and smooth transitions.",
        "Support beginners by avoiding jargon and including helpful examples."
    ]),
    backstory="Turns raw research insights into a complete paper draft.",
    llm=basic_llm,
    verbose=True,
)

draft_writer_task = Task(
    description="\n".join([
        "Input is: paper structure + research gap + related work.",
        "Use them to generate a coherent draft of the academic paper.",
        "Output in well-organized academic format (Intro, Method, etc.)."
    ]),
    expected_output="String containing the full paper draft.",
    output_json=DraftOutput,
    output_file=os.path.join(output_dir, "step_7_paper_draft.json"),
    agent=draft_writer_agent,
)

## `08` Crew

In [112]:
# Define the Crew
crew_agents = Crew(
    name="PublishMate Crew",
    
    description="A crew of agents designed to assist with academic research and paper writing.",

    agents=[trending_topics_agent, 
            recent_papers_agent, 
        #     paper_summarization_agent, 
        #     research_gap_agent, 
        #     paper_structure_agent, 
        #     related_work_agent, 
        #     draft_writer_agent
            ],
    

    tasks=[trending_topics_task, 
           recent_papers_task, 
        #    paper_summarization_task, 
        #    research_gap_task, 
        #    paper_structure_task, 
        #    related_work_task, 
        #    draft_writer_task
           ],
    # tools=[tavily_paper_search],
)

result = crew_agents.kickoff()
print(result)

# Agent: Trending Topics Identification Agent
## Task: you are an expert in a AI field to help beginner researchers in their writings .
Provide a list of 3 to 5 trending topics or articals with a brief description for each.
Focus on recent research interests supported by publication trends.
Output in JSON format with 'topics' as list of objects {name, description}.


# Agent: Trending Topics Identification Agent
## Final Answer: 
{
  "topics": [
    {
      "name": "Large Language Models (LLMs) and their limitations",
      "description": "Research is booming on improving LLMs' reasoning, factual accuracy, and mitigating biases.  Recent work focuses on techniques like chain-of-thought prompting, retrieval-augmented generation, and developing more robust evaluation metrics.  Conferences like NeurIPS and ICLR showcase numerous papers addressing these challenges, highlighting the ongoing efforts to make LLMs more reliable and trustworthy."
    },
    {
      "name": "Generative AI for Sci

🖇 AgentOps: Session Replay for default.session trace: https://app.agentops.ai/sessions?trace_id=9abefa5f05e37454a2228b2fdab4ac98


{'topic_papers': {}}
